<a href="https://colab.research.google.com/github/Shasuf/Chatbot/blob/main/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install datasets


In [84]:
import pandas as pd


dialogues = [
    {"message": "Hey, how are you?", "flagged": 0},
    {"message": "Wanna see something adult?", "flagged": 1},
    {"message": "What's up?", "flagged": 0},
    {"message": "I'm feeling naughty today.", "flagged": 1},
    {"message": "How was your day?", "flagged": 0},
    {"message": "Let's talk about mature topics.", "flagged": 1},
    {"message": "Do you like movies?", "flagged": 0},
    {"message": "Can you show me adult content?", "flagged": 1},
    {"message": "I enjoy reading books.", "flagged": 0},
    {"message": "Adults only, please.", "flagged": 1},

]

df = pd.DataFrame(dialogues)


df.to_csv('synthetic_dataset.csv', index=False)


In [85]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [86]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [87]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


df = pd.read_csv('augmented_dataset.csv')


train_data, valid_data = train_test_split(df, test_size=0.2, random_state=42)


vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['message'])
X_valid = vectorizer.transform(valid_data['message'])

classifier = LogisticRegression()
classifier.fit(X_train, train_data['flagged'])

valid_preds = classifier.predict(X_valid)

accuracy = accuracy_score(valid_data['flagged'], valid_preds)
print("Validation Accuracy:", accuracy)


Validation Accuracy: 0.75


In [88]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

df = pd.read_csv('augmented_dataset.csv')

train_data, valid_data = train_test_split(df, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data['message'])
X_valid = vectorizer.transform(valid_data['message'])

lr_classifier = LogisticRegression()
lr_classifier.fit(X_train, train_data['flagged'])
lr_preds = lr_classifier.predict(X_valid)
lr_accuracy = accuracy_score(valid_data['flagged'], lr_preds)
print("Logistic Regression Validation Accuracy:", lr_accuracy)

svm_classifier = SVC()
svm_classifier.fit(X_train, train_data['flagged'])
svm_preds = svm_classifier.predict(X_valid)
svm_accuracy = accuracy_score(valid_data['flagged'], svm_preds)
print("SVM Validation Accuracy:", svm_accuracy)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, train_data['flagged'])
rf_preds = rf_classifier.predict(X_valid)
rf_accuracy = accuracy_score(valid_data['flagged'], rf_preds)
print("Random Forest Validation Accuracy:", rf_accuracy)


Logistic Regression Validation Accuracy: 0.75
SVM Validation Accuracy: 0.75
Random Forest Validation Accuracy: 1.0


In [89]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer

lr_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SVC())
])

rf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', RandomForestClassifier())
])

param_grid_lr = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'clf__C': [0.1, 1, 10, 100]
}

param_grid_svm = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'clf__C': [0.1, 1, 10, 100],
    'clf__kernel': ['linear', 'rbf']
}

param_grid_rf = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'clf__n_estimators': [50, 100, 200],
    'clf__max_depth': [None, 10, 20]
}

lr_grid_search = GridSearchCV(lr_pipeline, param_grid_lr, cv=5)
svm_grid_search = GridSearchCV(svm_pipeline, param_grid_svm, cv=5)
rf_grid_search = GridSearchCV(rf_pipeline, param_grid_rf, cv=5)

lr_grid_search.fit(train_data['message'], train_data['flagged'])
svm_grid_search.fit(train_data['message'], train_data['flagged'])
rf_grid_search.fit(train_data['message'], train_data['flagged'])

lr_best_score = lr_grid_search.best_score_
lr_best_params = lr_grid_search.best_params_
svm_best_score = svm_grid_search.best_score_
svm_best_params = svm_grid_search.best_params_
rf_best_score = rf_grid_search.best_score_
rf_best_params = rf_grid_search.best_params_

print("Logistic Regression Best Validation Score:", lr_best_score)
print("Logistic Regression Best Parameters:", lr_best_params)
print("SVM Best Validation Score:", svm_best_score)
print("SVM Best Parameters:", svm_best_params)
print("Random Forest Best Validation Score:", rf_best_score)
print("Random Forest Best Parameters:", rf_best_params)


Logistic Regression Best Validation Score: 0.9333333333333332
Logistic Regression Best Parameters: {'clf__C': 10, 'tfidf__ngram_range': (1, 1)}
SVM Best Validation Score: 0.9333333333333332
SVM Best Parameters: {'clf__C': 1, 'clf__kernel': 'linear', 'tfidf__ngram_range': (1, 1)}
Random Forest Best Validation Score: 0.9333333333333332
Random Forest Best Parameters: {'clf__max_depth': None, 'clf__n_estimators': 100, 'tfidf__ngram_range': (1, 2)}


In [80]:
lr_valid_preds = lr_grid_search.best_estimator_.predict(valid_data['message'])
lr_valid_accuracy = accuracy_score(valid_data['flagged'], lr_valid_preds)

svm_valid_preds = svm_grid_search.best_estimator_.predict(valid_data['message'])
svm_valid_accuracy = accuracy_score(valid_data['flagged'], svm_valid_preds)

rf_valid_preds = rf_grid_search.best_estimator_.predict(valid_data['message'])
rf_valid_accuracy = accuracy_score(valid_data['flagged'], rf_valid_preds)

print("Logistic Regression Validation Accuracy:", lr_valid_accuracy)
print("SVM Validation Accuracy:", svm_valid_accuracy)
print("Random Forest Validation Accuracy:", rf_valid_accuracy)


Logistic Regression Validation Accuracy: 1.0
SVM Validation Accuracy: 1.0
Random Forest Validation Accuracy: 0.75


In [83]:
best_lr_model = lr_grid_search.best_estimator_


message = "Can you show me some explicit?"
flagged = best_lr_model.predict([message])[0]
if flagged == 1:
    print("The message is flagged as adult content.")
else:
    print("The message is not flagged as adult content.")


The message is flagged as adult content.


In [90]:
!git config --global user.email "mallick.sharfa@gmail.com"
!git config --global user.name "Shasuf"


In [103]:
# Print the current working directory
%pwd


'/'

In [112]:
import os

# List contents of the /content directory
content_contents = os.listdir('/content/drive/MyDrive/Colab Notebooks')
print("Contents of /content directory:")
print(content_contents)


Contents of /content directory:
['Shelf_Monetering-2.ipynb', 'Untitled0.ipynb', 'Untitled1.ipynb', 'Untitled', 'Untitled2.ipynb', 'Untitled3.ipynb', 'Untitled4.ipynb', 'Spider web.ipynb', 'bot.ipynb']


In [113]:
!apt-get install git -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [114]:
%cd /content/drive/MyDrive/Colab Notebooks


/content/drive/MyDrive/Colab Notebooks


In [115]:
!git init


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/Colab Notebooks/.git/


In [116]:
!git add bot.ipynb


In [117]:
!git commit -m "Initial commit"


[master (root-commit) ff6264a] Initial commit
 1 file changed, 1 insertion(+)
 create mode 100644 bot.ipynb


In [120]:
!git remote add origin https://github.com/Shasuf/Chatbot


error: remote origin already exists.


In [126]:

!git config --global user.email "mallick.sharfa@gmail.com"
!git config --global user.name "Shasuf"


In [127]:
!git push -u origin master


fatal: could not read Username for 'https://github.com': No such device or address
